In [1]:
# This notebook starts as a continuation of the previous notebook (1.5_Process_data_embeddings.ipynb)

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

df = pd.read_csv('1_Using_Chatbot_API/mini-llama-articles-with-embeddings.csv', sep='\t', encoding='utf-8')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   chunk      174 non-null    object
 1   embedding  174 non-null    object
dtypes: object(2)
memory usage: 2.8+ KB


In [3]:
# Our list is stored as strings, so we need to convert them to lists

import ast

df['embedding'] = df['embedding'].apply(ast.literal_eval)

In [4]:
from dotenv import load_dotenv
import os
import openai
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
# Set the API key for the OpenAI client
openai.api_key = api_key

g_api_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=g_api_key)

In [6]:
# Generate embeddings for the user input based on OpenAI's Ada model.
from openai import OpenAI

client = OpenAI()

# Defining a function that converts a text to embedding vector using OpenAI's Ada model.
def get_embedding(text):
  try:
    # Remove newlines
    text = text.replace("\n", " ")
    res = client.embeddings.create(input=[text], model="text-embedding-3-small")

    return res.data[0].embedding

  except:
        return None

In [7]:
# Get the top 3 chunks for each question

QUESTION = "How many parameters LLaMA2 model has?"
QUESTION_emb = get_embedding(QUESTION)

# The similarity between the questions and each part of the essay.
cosine_similarities = cosine_similarity([QUESTION_emb], df['embedding'].tolist())

number_of_chunks_to_retrieve = 3

# Sort and find the index of N highest scored chunks
indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]
print(indices)

[114  75  89]


In [17]:
# Use the Gemini API to answer the questions based on the retrieved pieces of text.
try:
    # Formulating the system prompt and condition the model to answer only AI-related questions.
    system_prompt = (
        "You are an assistant and expert in answering questions from a chunks of content. "
        "Only answer AI-related question, else say that you cannot answer this question."
    )

    # Create a user prompt with the user's question
    prompt = (
        "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
    # Add the retrieved pieces of text to the prompt.
    prompt = prompt.format("".join(df.chunk[indices]), QUESTION)

    model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)
    
    result = model.generate_content(prompt,request_options={"timeout": 1000},)
    res = result.candidates[0].content.parts[0].text
    print(res)

except Exception as e:
    print(f"An error occurred: {e}")

LLaMA 2 comes in four sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.



In [18]:
# Answering the question after Knowledge cutoff without any context.

QUESTION = "How many parameters LLaMA 3.1 model has?"

# Formulating the system prompt
system_prompt = "You are an assistant and expert in answering questions."

# Combining the system prompt with the user's question
prompt = "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"
prompt = prompt.format(QUESTION)

model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)

#Gemini API call
result = model.generate_content(prompt,request_options={"timeout": 1000},)
res = result.candidates[0].content.parts[0].text
print(res)

The exact number of parameters for the LLaMA 3.1 model isn't publicly released by Meta.  Meta has not disclosed the parameter counts for the LLaMA 3 models in the same way they did for the original LLaMA models.



In [19]:
# Consider this as a retrieved chunk
# https://ai.meta.com/blog/meta-llama-3-1/
Example_chunk = """
Introducing Llama 3.1 Llama 3.1 405B is the first openly available model that rivals the top AI models when it comes to state-of-the-art capabilities in general knowledge, steerability, math, tool use, and multilingual translation. With the release of the 405B model, we’re poised to supercharge innovation—with unprecedented opportunities for growth and exploration. We believe the latest generation of Llama will ignite new applications and modeling paradigms, including synthetic data generation to enable the improvement and training of smaller models, as well as model distillation—a capability that has never been achieved at this scale in open source.
As part of this latest release, we’re introducing upgraded versions of the 8B and 70B models. These are multilingual and have a significantly longer context length of 128K, state-of-the-art tool use, and overall stronger reasoning capabilities. This enables our latest models to support advanced use cases, such as long-form text summarization, multilingual conversational agents, and coding assistants. We’ve also made changes to our license, allowing developers to use the outputs from Llama models—including the 405B—to improve other models. True to our commitment to open source, starting today, we’re making these models available to the community for download on llama.meta.com and Hugging Face and available for immediate development on our broad ecosystem of partner platforms. Model evaluations
For this release, we evaluated performance on over 150 benchmark datasets that span a wide range of languages. In addition, we performed extensive human evaluations that compare Llama 3.1 with competing models in real-world scenarios. Our experimental evaluation suggests that our flagship model is competitive with leading foundation models across a range of tasks, including GPT-4, GPT-4o, and Claude 3.5 Sonnet. Additionally, our smaller models are competitive with closed and open models that have a similar number of parameters.
Model Architecture As our largest model yet, training Llama 3.1 405B on over 15 trillion tokens was a major challenge. To enable training runs at this scale and achieve the results we have in a reasonable amount of time, we significantly optimized our full training stack and pushed our model training to over 16 thousand H100 GPUs, making the 405B the first Llama model trained at this scale.
To address this, we made design choices that focus on keeping the model development process scalable and straightforward. We opted for a standard decoder-only transformer model architecture with minor adaptations rather than a mixture-of-experts model to maximize training stability.
We adopted an iterative post-training procedure, where each round uses supervised fine-tuning and direct preference optimization. This enabled us to create the highest quality synthetic data for each round and improve each capability’s performance.
Compared to previous versions of Llama, we improved both the quantity and quality of the data we use for pre- and post-training. These improvements include the development of more careful pre-processing and curation pipelines for pre-training data, the development of more rigorous quality assurance, and filtering approaches for post-training data.
"""

In [20]:
# Ansering the question after Knowledge cutoff using the example context.
QUESTION = "How many parameters LLaMA 3.1 model has?"

# Formulating the system prompt
system_prompt = (
        "You are an assistant and expert in answering questions from a chunks of content. "
        "Only answer AI-related question, else say that you cannot answer this question."
    )
# Combining the system prompt with the user's question
prompt = (
        "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
prompt = prompt.format(Example_chunk, QUESTION)

model = genai.GenerativeModel(model_name= "gemini-1.5-flash", system_instruction=system_prompt)

#Gemini API call
result = model.generate_content(prompt,request_options={"timeout": 1000},)
res = result.candidates[0].content.parts[0].text
print(res)

The provided text mentions Llama 3.1 405B, indicating a model with 405 billion parameters.  It also mentions upgraded 8B and 70B parameter models.

